In [1]:
import numpy as np
f = open('day_11_input.txt')
input_list = f.read().split('\n')
input_array = np.array([list(item) for item in input_list])

In [2]:
input_array

array([['L', 'L', 'L', ..., 'L', 'L', 'L'],
       ['L', 'L', 'L', ..., 'L', '.', 'L'],
       ['L', 'L', 'L', ..., 'L', 'L', 'L'],
       ...,
       ['.', 'L', 'L', ..., '.', 'L', 'L'],
       ['L', 'L', 'L', ..., 'L', 'L', 'L'],
       ['L', 'L', 'L', ..., 'L', 'L', 'L']], dtype='<U1')

In [3]:
def day_11_part_1():

    #This function will generate and find the neighbors for the given inputs
    def find_neighbors(x_coordinate, y_coordinate, inp_array):
        
        #We generate the x and y neighbors that are valid, that is that their indices are within the array
        valid_x_neighbors = [x_coordinate + x for x in range(-1, 2) if (x_coordinate + x) >= 0 and (x_coordinate + x) < len(inp_array[0])]
        valid_y_neighbors = [y_coordinate + y for y in range(-1, 2) if (y_coordinate + y) >= 0 and (y_coordinate + y) < len(inp_array)]
        
        #Then, we generate the cartesian product of the x and y neighbors, removing for the cell itself
        neighbor_coordinates = [[x, y] for x in valid_x_neighbors for y in valid_y_neighbors if not ((x == x_coordinate) and (y == y_coordinate))]
        neighbor_list = []

        #For each coordinate, append the value found at the input array to the list
        for x_neighbor_coordinate, y_neighbor_coordinate in neighbor_coordinates:
            neighbor_list.append(inp_array[y_neighbor_coordinate, x_neighbor_coordinate])
        
        return np.array(neighbor_list)
    
    #This function will step through each "generation"
    def generation(inp_array):
        #Create a copy of the input array that will be mutated and set a counter tracking number of changes to zero
        array_copy = inp_array.copy()
        counter = 0
        #Iterate on each row while keeping track of the y index
        for y_index,row in enumerate(inp_array):
            #Iterate on each column while keeping track of the x index
            for x_index,item in enumerate(row):
                #Get the list of neighbors for the given cell
                neighbor_array = find_neighbors(x_index, y_index, inp_array)

                #Apply the generation rules. Each time there's a change, add 1 to the counter
                if item == "L" and ("#" not in neighbor_array):
                    array_copy[y_index, x_index] = "#"
                    counter += 1
                elif item == "#" and len(neighbor_array[neighbor_array == "#"]) >= 4:
                    array_copy[y_index, x_index] = "L"
                    counter += 1
        
        return array_copy, counter
    
    #Create a copy of the day 11 input and set a counter to 1, representing the amount of changes made in each generation
    array_copy = input_array.copy()
    counter = 1
    #Break as soon as a generation makes no change
    while counter > 0:
        array_copy, counter = generation(array_copy)
    
    #Return how many seats are occupied
    return len(array_copy[array_copy == '#'])

In [4]:
day_11_part_1()

2238

In [5]:
#Generate a 2D array where each entry contains a list of the valid neighbors to be found. This approach is significantly faster than the one taken above
def generate_neighbor_coordinates():
    #Hoist a variable representing the full neighbor array
    full_array = []

    #Iterate on each row, keeping track of the y index
    for y, row in enumerate(input_array):
        #Hoist a variable representing the inputs for each row
        new_row = []
        #Iterate on each column, keeping track of the x index
        for x, item in enumerate(row):
            #Hoist a variable that will keep the list of the coordinates of the neighbors for the given cell
            neighbor_coordinate_list = []

            #For each direction, create a counter that will keep going until it either collides with an empty seat or goes out of bounds. Record the collisions with empty seats
            #west
            counter = 1
            while (x - counter) >= 0:
                candidate = input_array[y, x - counter]
                if candidate != '.':
                    neighbor_coordinate_list.append([x - counter, y])
                    break
                else:
                    counter += 1
                
            
            #east
            counter = 1
            while (x + counter) < len(input_array[0]):
                candidate = input_array[y, x + counter]
                if candidate != '.':
                    neighbor_coordinate_list.append([x + counter, y])
                    break
                else:
                    counter += 1
            
            #north
            counter = 1
            while (y - counter) >= 0:
                candidate = input_array[y - counter, x]
                if candidate != '.':
                    neighbor_coordinate_list.append([x, y - counter])
                    break
                else:
                    counter += 1
            
            #south
            counter = 1
            while (y + counter) < len(input_array):
                candidate = input_array[y + counter, x]
                if candidate != '.':
                    neighbor_coordinate_list.append([x, y + counter])
                    break
                else:
                    counter += 1

            #northwest
            counter = 1
            while ((x - counter) >= 0) and ((y - counter) >= 0):
                candidate = input_array[y - counter, x - counter]
                if candidate != '.':
                    neighbor_coordinate_list.append([x - counter, y - counter])
                    break
                else:
                    counter += 1
            
            #northeast
            counter = 1
            while ((x + counter) < len(input_array[0])) and ((y - counter) >= 0):
                candidate = input_array[y - counter, x + counter]
                if candidate != '.':
                    neighbor_coordinate_list.append([x + counter, y - counter])
                    break
                else:
                    counter += 1
            
            #southwest
            counter = 1
            while ((x - counter) >= 0) and ((y + counter) < len(input_list)):
                candidate = input_array[y + counter, x - counter]
                if candidate != '.':
                    neighbor_coordinate_list.append([x - counter, y + counter])
                    break
                else:
                    counter += 1
            
            #southeast
            counter = 1
            while ((x + counter) < len(input_array[0])) and ((y + counter) < len(input_list)):
                candidate = input_array[y + counter, x + counter]
                if candidate != '.':
                    neighbor_coordinate_list.append([x + counter, y + counter])
                    break
                else:
                    counter += 1

            #Append the neighbor coordinate list to the new row
            new_row.append(neighbor_coordinate_list)
        #Turn each new_row into a numpy array, then append it to full_array
        full_array.append(np.array(new_row))
    #Turn full_array into a numpy array to allow for ease of access of each item
    return np.array(full_array)

In [6]:
full_neighbor_list = generate_neighbor_coordinates()

In [7]:
def day_11_part_2():

    #This function works equivalently to the one above, but uses the pre-built 2D array full_neighbor_list that contains the list for each appropriate cell
    def find_neighbors(x_coordinate, y_coordinate, inp_array):
        neighbor_indices = full_neighbor_list[y_coordinate, x_coordinate]
        neighbor_list = []
        for x_index, y_index in neighbor_indices:
            neighbor_list.append(inp_array[y_index, x_index])
        
        return np.array(neighbor_list)
    
    #This function works nearly identically to the one above, but has a tolerance for 5 seats instead of 4
    def generation(inp_array):
        array_copy = inp_array.copy()
        counter = 0
        for y_index,row in enumerate(inp_array):
            for x_index,item in enumerate(row):
                neighbor_array = find_neighbors(x_index, y_index, inp_array)
                if item == "L" and ("#" not in neighbor_array):
                    array_copy[y_index, x_index] = "#"
                    counter += 1
                elif item == "#" and len(neighbor_array[neighbor_array == "#"]) >= 5:
                    array_copy[y_index, x_index] = "L"
                    counter += 1
        
        return array_copy, counter
    
    #Again, works identically to the one above
    array_copy = input_array.copy()
    counter = 1
    while counter > 0:
        array_copy, counter = generation(array_copy)
    
    return len(array_copy[array_copy == "#"])

In [8]:
day_11_part_2()

2013